In [85]:
import os
import json
import pandas as pd
import ast
import numpy as np
import faiss
import torch
import math
import re

from typing import List, Dict, Any, Tuple
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from tqdm import tqdm
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
from rouge_score import rouge_scorer
from openai import OpenAI

In [7]:
pd.set_option('display.width', -1)
pd.set_option('max_colwidth', 1000)

In [8]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [9]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

### 1. Load and Flattion ConvFinQA JSON data
---

In [10]:
def convfinqadfloader(filepath: str, max_rows: int = 1000) -> pd.DataFrame:
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    if max_rows is not None:
        data = data[:max_rows]
    
    flattened_data = []
    
    for item in data:
        base_item = {
            'id': item.get('id'),
            'pre_text': ' '.join(item.get('pre_text', [])),
            'post_text': ' '.join(item.get('post_text', [])),
            'filename': item.get('filename'),
            'table': str(item.get('table')),
        }
        
        annotation = item.get('annotation', {})
        if isinstance(annotation, dict):
            dialogue_break = annotation.get('dialogue_break', [])
            turn_program = annotation.get('turn_program', [])
            qa_split = annotation.get('qa_split', [])
            exe_ans_list = annotation.get('exe_ans_list', [])
            for idx in range(len(dialogue_break)):
                turn_data = {
                    'dialogue_text': dialogue_break[idx] if idx < len(dialogue_break) else None,
                    'turn_program': turn_program[idx] if idx < len(turn_program) else None,
                    'qa_split': qa_split[idx] if idx < len(qa_split) else None,
                    'execution_answer': exe_ans_list[idx] if idx < len(exe_ans_list) else None,
                    'turn_index': idx
                }
                combined_data = {**base_item, **turn_data}
                flattened_data.append(combined_data)
        
        if 'qa' in item:
            qa_data = {
                'question': item['qa'].get('question'),
                'answer': item['qa'].get('answer'),
                'explanation': item['qa'].get('explanation'),
                'program': item['qa'].get('program'),
                'execution_answer': item['qa'].get('exe_ans'),
                'turn_index': 0
            }
            combined_data = {**base_item, **qa_data}
            flattened_data.append(combined_data)
    
    df = pd.DataFrame(flattened_data)
    
    numeric_columns = ['turn_index', 'qa_split', 'execution_answer']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df


In [11]:
# Load data
df = convfinqadfloader("data/convfinqatrain.json")
print("Dataset loaded. Number of examples:", len(df))

Dataset loaded. Number of examples: 4378


### 2. Create Combined Text Field
---

In [12]:
def create_combined_text(row):
    """
    Combine key text fields to form a context string.
    Uses 'pre_text', 'dialogue_text', 'post_text', 'execution_answer', and 'question' (if available).
    """
    texts = []
    if pd.notnull(row.get('pre_text')):
        texts.append("Pre-Text: " + row['pre_text'])
    if pd.notnull(row.get('dialogue_text')):
        texts.append("Dialogue: " + row['dialogue_text'])
    if pd.notnull(row.get('post_text')):
        texts.append("Post-Text: " + row['post_text'])
    if pd.notnull(row.get('execution_answer')):
        texts.append("Execution Answer: " + str(row['execution_answer']))
    if pd.notnull(row.get('question')):
        texts.append("Question: " + row['question'])
    return " | ".join(texts)

df['combined_text'] = df.apply(create_combined_text, axis=1)

### 3. Chunking
---

In [13]:
def chunk_text(text, chunk_size=200, chunk_overlap=100):
    """
    Split a text into chunks of words with specified chunk size and overlap.
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        chunk = " ".join(words[start: start + chunk_size])
        chunks.append(chunk)
        if start + chunk_size >= len(words):
            break
        start += (chunk_size - chunk_overlap)
    return chunks

In [14]:
# Use a subset of the data (e.g., first 500 documents)
subset_df = df.iloc[:500].copy()

In [15]:
chunk_records = []
for idx, row in subset_df.iterrows():
    chunks = chunk_text(row['combined_text'], chunk_size=200, chunk_overlap=100)
    for i, chunk in enumerate(chunks):
        chunk_records.append({
            "doc_id": row['id'],
            "chunk_text": chunk,
            "turn_index": row.get('turn_index', None)
        })

In [16]:
chunk_df = pd.DataFrame(chunk_records)
print(f"Created {len(chunk_df)} chunks from {len(subset_df)} documents.")

Created 3019 chunks from 500 documents.


### 4. OpenAI Embeddings
---

In [17]:
# Use OpenAI's ada-002 embeddings
def get_embedding(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=[text], model=model)
    return np.array(response.data[0].embedding, dtype=np.float32)

In [18]:
def compute_embeddings(texts, engine="text-embedding-ada-002"):
    embeddings = []
    for text in texts:
        emb = get_embedding(text, model=engine)
        embeddings.append(emb)
    return np.vstack(embeddings)

Create embeddings on chunked documents ~ 20 minutes

In [19]:
# Compute embeddings for all chunks
chunk_texts = chunk_df['chunk_text'].tolist()
print("Computing embeddings for {} chunks...".format(len(chunk_texts)))
document_embeddings = compute_embeddings(chunk_texts)
embedding_dim = document_embeddings.shape[1]

Computing embeddings for 3019 chunks...


In [20]:
print("Done!")

Done!


### 5. Create FAISS Index
---

In [21]:
faiss.normalize_L2(document_embeddings)

In [22]:
index = faiss.IndexFlatIP(embedding_dim)
index.add(document_embeddings)
print("FAISS index built with {} vectors.".format(index.ntotal))

FAISS index built with 3019 vectors.


In [23]:
def query_dataset(query, top_n=3, engine="text-embedding-ada-002"):
    query_embedding = get_embedding(query, model=engine).reshape(1, -1)
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, top_n)
    results = chunk_df.iloc[indices[0]].copy()
    results['score'] = distances[0]
    return results

### 6. Generation with GPT-4o-mini
---

In [104]:
def generate_answer(query, context_docs, max_tokens=200, temperature=0.0, top_p=1.0):
    """
    Generate an answer by combining the query with retrieved context using GPT-4o-mini.
    The system prompt instructs the model to act as a friendly financial analyst bot that does not
    make up answers and only uses the provided data to answer numerical questions.
    """
    
    # Combine the retrieved context documents into a single string.
    context = "\n".join(context_docs)
    
    # Create a messages list with a system prompt and the user's prompt.
    messages = [
         {
             "role": "system",
             "content": "You are a friendly financial analyst bot. When given a question and context, you respond with only a single floating point number (and nothing else), which is the answer based solely on the provided data."
         },
         {
             "role": "user",
             "content": f"Question: {query}\nContext: {context}\nAnswer (only a number):"
         }
    ]
    response = openai.chat.completions.create(
         model="gpt-4o-mini",
         messages=messages,
         max_tokens=max_tokens,
         temperature=temperature,
         top_p=top_p,
         n=1,
         stop=["\n"]
    )
    return response.choices[0].message.content.strip()


### 7. Train Test Split and Evaluation Setup
---

In [105]:
# For evaluation, use only examples with valid 'question' and 'execution_answer'
eval_df = df[(df['execution_answer'].notna()) & (df['question'].notna())].copy()
print(f"Evaluation dataset size: {len(eval_df)}")

Evaluation dataset size: 714


In [106]:
# 80:20 train-test split (we'll use the test set for evaluation)
_, test_df = train_test_split(eval_df, test_size=0.2, random_state=42)
print(f"Test set size: {len(test_df)}")

Test set size: 143


In [107]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

### 8. Evaluation Loop (Numeric Answers)
---

In [108]:
total_correct = 0
total_retrieval_precision = 0.0
total_rouge1 = 0.0
total_rouge2 = 0.0
total_rougeL = 0.0
num_evaluated = 0
tolerance = 1e-2  # Tolerance for numeric comparison

In [109]:
for idx, row in test_df.iterrows():
    query_text = row['question']
    
    # Convert ground truth to float
    try:
        ground_truth = float(row['execution_answer'])
    except Exception as e:
        continue  # Skip if conversion fails
    
    retrieved = query_dataset(query_text, top_n=3)
    context_docs = retrieved['chunk_text'].tolist()
    
    generated_text = generate_answer(query_text, context_docs, max_tokens=200, temperature=0.0, top_p=1.0).strip()
    
    try:
        generated_val = float(generated_text)
    except Exception as e:
        generated_val = None
    
    if generated_val is not None and math.isclose(generated_val, ground_truth, rel_tol=tolerance):
        total_correct += 1
        
    relevant_count = sum(1 for x in retrieved['doc_id'] if x == row['id'])
    retrieval_precision = relevant_count / len(retrieved) if len(retrieved) > 0 else 0.0
    total_retrieval_precision += retrieval_precision
    
    gt_str = f"{ground_truth:.2f}"
    gen_str = f"{generated_val:.2f}" if generated_val is not None else ""
    rouge_scores = scorer.score(gt_str, gen_str)
    total_rouge1 += rouge_scores['rouge1'].fmeasure
    total_rouge2 += rouge_scores['rouge2'].fmeasure
    total_rougeL += rouge_scores['rougeL'].fmeasure
    
    num_evaluated += 1
    
    print(f"Example {idx}:")
    print(f"Question: {query_text}")
    print(f"Ground Truth: {ground_truth}")
    print(f"Generated: {generated_text} (parsed as {generated_val})")
    print(f"Retrieval Precision: {retrieval_precision:.3f}")
    print(f"ROUGE-1: {rouge_scores['rouge1'].fmeasure:.3f}, ROUGE-2: {rouge_scores['rouge2'].fmeasure:.3f}, ROUGE-L: {rouge_scores['rougeL'].fmeasure:.3f}")
    print("----------\n")

Example 698:
Question: what were total r&e expenses in millions for 2017 , 2016 and in 2015?
Ground Truth: 581.0
Generated: 72.0 (parsed as 72.0)
Retrieval Precision: 0.000
ROUGE-1: 0.500, ROUGE-2: 0.000, ROUGE-L: 0.500
----------

Example 2048:
Question: what is the total sublease revenue , in millions , from 2008-2010?
Ground Truth: 18.2
Generated: 31.2 (parsed as 31.2)
Retrieval Precision: 0.000
ROUGE-1: 0.500, ROUGE-2: 0.000, ROUGE-L: 0.500
----------

Example 230:
Question: by how much did total other income and expense decrease from 2008 to 2009?
Ground Truth: 0.47419
Generated: 465.0 (parsed as 465.0)
Retrieval Precision: 1.000
ROUGE-1: 0.000, ROUGE-2: 0.000, ROUGE-L: 0.000
----------

Example 1782:
Question: what was the difference in percentage cumulative total shareowners 2019 returns for united parcel service inc . compared to the standard & poor's 500 index for the five years ended 12/31/2014?
Ground Truth: 0.1874
Generated: -0.0142 (parsed as -0.0142)
Retrieval Precision: 

In [110]:
accuracy = total_correct / num_evaluated if num_evaluated > 0 else 0.0
avg_retrieval_precision = total_retrieval_precision / num_evaluated if num_evaluated > 0 else 0.0
avg_rouge1 = total_rouge1 / num_evaluated if num_evaluated > 0 else 0.0
avg_rouge2 = total_rouge2 / num_evaluated if num_evaluated > 0 else 0.0
avg_rougeL = total_rougeL / num_evaluated if num_evaluated > 0 else 0.0

print("Evaluation Metrics:")
print(f"Accuracy (Exact Match within tolerance): {accuracy:.3f}")
print(f"Average Retrieval Precision: {avg_retrieval_precision:.3f}")
print(f"Average ROUGE-1: {avg_rouge1:.3f}")
print(f"Average ROUGE-2: {avg_rouge2:.3f}")
print(f"Average ROUGE-L: {avg_rougeL:.3f}")

Evaluation Metrics:
Accuracy (Exact Match within tolerance): 0.063
Average Retrieval Precision: 0.105
Average ROUGE-1: 0.371
Average ROUGE-2: 0.105
Average ROUGE-L: 0.371


### 9. Test the Generate answe function on a single query and execute turn program
---

In [111]:
test_query = "what is the net cash from operating activities in 2009?"
test_retrieved = query_dataset(test_query, top_n=3)
test_context_docs = test_retrieved['chunk_text'].tolist()
test_generated_answer = generate_answer(test_query, test_context_docs, max_tokens=200, temperature=0.0, top_p=1.0)
print("\nTest Generated Answer for query:")
print(f"Question: {test_query}")
print(f"Generated Answer: {test_generated_answer}")


Test Generated Answer for query:
Question: what is the net cash from operating activities in 2009?
Generated Answer: 206588.0


In [112]:
# Additionally, attempt to execute the turn program from the top retrieved document, if available.
top_doc_id = test_retrieved.iloc[0]['doc_id']
# Retrieve the original example from the full dataframe
top_example = df[df['id'] == top_doc_id].iloc[0]
if 'turn_program' in top_example and pd.notnull(top_example['turn_program']):
    cur_program = top_example['turn_program']
    print("\nRetrieved 'turn_program' for execution:")
    print(cur_program)
    try:
        executed_answer = eval(cur_program)
        print("Executed Answer:", executed_answer)
    except Exception as e:
        print("Error executing turn_program:", e)
else:
    print("No 'turn_program' available for the selected example.")


Retrieved 'turn_program' for execution:
206588
Executed Answer: 206588
